In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from src.projection import TverskyProjection

In [ ]:
class CNNFeatureExtractor(nn.Module):
    """A simple CNN base shared by both models for fair comparison."""
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.flattened_size = 32 * 7 * 7

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.flattened_size)
        return x

In [3]:
class BaselineNet(nn.Module):
    """Standard CNN with a Linear classification head."""
    def __init__(self):
        super().__init__()
        self.features = CNNFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(self.features.flattened_size, 128),
            nn.ReLU(),
            nn.Linear(128, 10) # 10 classes for MNIST
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
class TverskyNet(nn.Module):
    """CNN with our contextual Tversky classification head."""
    def __init__(self, num_tversky_features=64):
        super().__init__()
        self.features = CNNFeatureExtractor()
        # The first linear layer is the same as the baseline
        self.fc1 = nn.Linear(self.features.flattened_size, 128)

        self.tversky_head = TverskyProjection(
            input_dim=128,
            output_dim=10,
            num_features=num_tversky_features
        )

    def forward(self, x):
        x = self.features(x)
        x = F.relu(self.fc1(x))
        x = self.tversky_head(x)
        return x

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [ ]:
BATCH_SIZE = 64
EPOCHS = 3 
LR = 0.001
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f"Using device: {device}")

# Data Loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST mean and std
])
train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("="*40)
print("         TRAINING BASELINE MODEL")
print("="*40)
baseline_model = BaselineNet().to(device)
optimizer = optim.Adam(baseline_model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    train(baseline_model, device, train_loader, optimizer, epoch)
baseline_accuracy = test(baseline_model, device, test_loader)

print("\n" + "="*40)
print("          TRAINING TVERSKY MODEL")
print("="*40)
tversky_model = TverskyNet().to(device)
optimizer = optim.Adam(tversky_model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    train(tversky_model, device, train_loader, optimizer, epoch)
tversky_accuracy = test(tversky_model, device, test_loader)
    
print("\n" + "="*40)
print("             FINAL RESULTS")
print("="*40)
print(f"Baseline Model Accuracy: {baseline_accuracy:.2f}%")
print(f"TverskyNet Model Accuracy: {tversky_accuracy:.2f}%")

Using device: cpu
         TRAINING BASELINE MODEL
Train Epoch: 1 [0/60000] Loss: 2.292212
Train Epoch: 1 [12800/60000] Loss: 0.243280
Train Epoch: 1 [25600/60000] Loss: 0.209473
Train Epoch: 1 [38400/60000] Loss: 0.077867
Train Epoch: 1 [51200/60000] Loss: 0.025532
Train Epoch: 2 [0/60000] Loss: 0.109538
Train Epoch: 2 [12800/60000] Loss: 0.079502
Train Epoch: 2 [25600/60000] Loss: 0.013300
Train Epoch: 2 [38400/60000] Loss: 0.060613
Train Epoch: 2 [51200/60000] Loss: 0.029926
Train Epoch: 3 [0/60000] Loss: 0.038683
Train Epoch: 3 [12800/60000] Loss: 0.021222
Train Epoch: 3 [25600/60000] Loss: 0.009130
Train Epoch: 3 [38400/60000] Loss: 0.010784
Train Epoch: 3 [51200/60000] Loss: 0.006513

Test set: Average loss: 0.0395, Accuracy: 9864/10000 (98.64%)


          TRAINING TVERSKY MODEL
Train Epoch: 1 [0/60000] Loss: 2.301561
Train Epoch: 1 [12800/60000] Loss: 0.395835
Train Epoch: 1 [25600/60000] Loss: 0.206514
Train Epoch: 1 [38400/60000] Loss: 0.255627
Train Epoch: 1 [51200/60000] Lo